In [4]:
import concurrent.futures
import threading
import time
from collections import deque

# Graph class
class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.graph = {i: [] for i in range(vertices)}  # adjacency list

    def add_edge(self, u, v):
        self.graph[u].append(v)
        self.graph[v].append(u)

    def get_neighbors(self, vertex):
        return self.graph[vertex]

# Parallel BFS
def parallel_bfs(graph, start_vertex):
    visited = [False] * graph.vertices
    queue = deque([start_vertex])
    visited[start_vertex] = True
    result = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        while queue:
            current_vertex = queue.popleft()
            result.append(current_vertex)
            futures = []
            for neighbor in graph.get_neighbors(current_vertex):
                if not visited[neighbor]:
                    visited[neighbor] = True
                    futures.append(executor.submit(queue.append, neighbor))
            for future in concurrent.futures.as_completed(futures):
                future.result()
    return result

# Parallel DFS
def parallel_dfs(graph, start_vertex):
    visited = [False] * graph.vertices
    result = []
    lock = threading.Lock()

    def dfs(v):
        with lock:
            if visited[v]:
                return
            visited[v] = True
            result.append(v)
        futures = []
        for neighbor in graph.get_neighbors(v):
            with lock:
                if not visited[neighbor]:
                    futures.append(executor.submit(dfs, neighbor))
        for future in concurrent.futures.as_completed(futures):
            future.result()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        dfs(start_vertex)
    return result

# Measure time
def measure_time(function, graph, start_vertex):
    start_time = time.time()
    result = function(graph, start_vertex)
    end_time = time.time()
    return result, end_time - start_time

# Main function
def main():
    g = Graph(6)
    g.add_edge(0, 1)
    g.add_edge(0, 2)
    g.add_edge(1, 3)
    g.add_edge(1, 4)
    g.add_edge(2, 5)

    start_vertex = 0
    print(f"Starting from vertex: {start_vertex}")

    bfs_result, bfs_time = measure_time(parallel_bfs, g, start_vertex)
    print(f"\nParallel BFS result: {bfs_result}")
    print(f"Execution Time: {bfs_time:.6f} seconds")

    dfs_result, dfs_time = measure_time(parallel_dfs, g, start_vertex)
    print(f"\nParallel DFS result: {dfs_result}")
    print(f"Execution Time: {dfs_time:.6f} seconds")

if __name__ == "__main__":
    main()


Starting from vertex: 0

Parallel BFS result: [0, 1, 2, 3, 4, 5]
Execution Time: 0.003989 seconds

Parallel DFS result: [0, 1, 2, 3, 4, 5]
Execution Time: 0.004876 seconds
